In [1]:
#Importation des bibliothèques 
import pandas as pd
import openrouteservice as ors
import numpy as np
import folium 

In [2]:
#fonction de construction de laa matrice des distabces 
def Matrice_des_distances(path):#elle prend le path du csv comme inoput
    file=pd.read_csv(path)
    f=file.to_numpy()
    client = ors.Client(key='5b3ce3597851110001cf624803bbfd9756c04be9aed554b68d5a1fd2')#clé du client de l'api openrouteservice
    d=[]
    for i in range (len(f)):#extraction des coordonnées deouis le csv 
        d+=[[f[i][2],f[i][3]]]
    x=[]
    for s in d:
        tuple(s)
        x.append(tuple(s)) #construction d'une liste qui contient des tuples contenant les coordonnées 
    myMATRIX = [x] + [[e] + [0] * (len(x)-1) for e in x[0:]] #construction d'une matrice vide , et qui à comme header et première colonne la liste x 
    lst=myMATRIX()
    for i in range (1,len(lst)):#on ajoute des zéros à la matrice pour fixier sa taille et avoir (len(nombre des locations)+1)^2
        lst[i].append(0)
    lst[0].insert(0,0)
    List=lst
    for i in range (1,len(lst[0])):#boucle qui fait appel au l'API et remplit la matrice construite par les distances calculées par le client 
        for j in range (1,len(lst)):       
            List[i][j]=(client.directions(tuple([List[i][0],List[0][j]]))).get("routes")[0].get("summary").get("distance")
    return List

In [3]:
L=Matrice_des_distances("C:\\Users\\WIAM\\OneDrive\\Documents\\ors.csv")

In [4]:
#Matrice qui cert d'enlever le header et la première colonne depuis la matrice des distances obtenue 
def generer_matrice_pure(List):
    T=List[1:]
    for i in range(len(T)):
        T[i].pop(0)
    return(T)
MP=generer_matrice_pure(L)
n=np.array(MP)
n[n==None]=0


In [7]:
import math

maxsize = float('inf')
    
def BranchandBound(dist):
    global adj
    global N
    global final_path
    global visited
    global final_res
    adj=dist
    N = len(adj)
    final_path = [None] * (N + 1)
    visited = [False] * N
    final_res = maxsize
    TSP(adj)
    return final_path

def bandb(lisst):
    q=BranchandBound(n)
    q = [x+1 for x in q]
    return q
    
    
# Function to copy temporary solution
# to the final solution
def copyToFinal(curr_path):
    final_path[:N + 1] = curr_path[:]
    final_path[N] = curr_path[0]


# Function to find the minimum edge cost
# having an end at the vertex i
def firstMin(adj, i):
    min = maxsize
    for k in range(N):
        if adj[i][k] < min and i != k:
            min = adj[i][k]

    return min


# function to find the second minimum edge
# cost having an end at the vertex i
def secondMin(adj, i):
    first, second = maxsize, maxsize
    for j in range(N):
        if i == j:
            continue
        if adj[i][j] <= first:
            second = first
            first = adj[i][j]

        elif (adj[i][j] <= second and
              adj[i][j] != first):
            second = adj[i][j]

    return second


# function that takes as arguments:
# curr_bound -> lower bound of the root node
# curr_weight-> stores the weight of the path so far
# level-> current level while moving
# in the search space tree
# curr_path[] -> where the solution is being stored
# which would later be copied to final_path[]
def TSPRec(adj, curr_bound, curr_weight,
           level, curr_path, visited):
    global final_res

    # base case is when we have reached level N
    # which means we have covered all the nodes once
    if level == N:

        # check if there is an edge from
        # last vertex in path back to the first vertex
        if adj[curr_path[level - 1]][curr_path[0]] != 0:

            # curr_res has the total weight
            # of the solution we got
            curr_res = curr_weight + adj[curr_path[level - 1]] \
                [curr_path[0]]
            if curr_res < final_res:
                copyToFinal(curr_path)
                final_res = curr_res
        return

    # for any other level iterate for all vertices
    # to build the search space tree recursively
    for i in range(N):

        # Consider next vertex if it is not same
        # (diagonal entry in adjacency matrix and
        # not visited already)
        if (adj[curr_path[level - 1]][i] != 0 and
                visited[i] == False):
            temp = curr_bound
            curr_weight += adj[curr_path[level - 1]][i]

            # different computation of curr_bound
            # for level 2 from the other levels
            if level == 1:
                curr_bound -= ((firstMin(adj, curr_path[level - 1]) +
                                firstMin(adj, i)) / 2)
            else:
                curr_bound -= ((secondMin(adj, curr_path[level - 1]) +
                                firstMin(adj, i)) / 2)

            # curr_bound + curr_weight is the actual lower bound
            # for the node that we have arrived on.
            # If current lower bound < final_res,
            # we need to explore the node further
            if curr_bound + curr_weight < final_res:
                curr_path[level] = i
                visited[i] = True

                # call TSPRec for the next level
                TSPRec(adj, curr_bound, curr_weight,
                       level + 1, curr_path, visited)

            # Else we have to prune the node by resetting
            # all changes to curr_weight and curr_bound
            curr_weight -= adj[curr_path[level - 1]][i]
            curr_bound = temp

            # Also reset the visited array
            visited = [False] * len(visited)
            for j in range(level):
                if curr_path[j] != -1:
                    visited[curr_path[j]] = True


# This function sets up final_path
def TSP(adj):
    # Calculate initial lower bound for the root node
    # using the formula 1/2 * (sum of first min +
    # second min) for all edges. Also initialize the
    # curr_path and visited array
    curr_bound = 0
    curr_path = [-1] * (N + 1)
    visited = [False] * N

    # Compute initial bound
    for i in range(N):
        curr_bound += (firstMin(adj, i) +
                       secondMin(adj, i))

    # Rounding off the lower bound to an integer
    curr_bound = math.ceil(curr_bound / 2)

    # We start at vertex 1 so the first vertex
    # in curr_path[] is 0
    visited[0] = True
    curr_path[0] = 0

    # Call to TSPRec for curr_weight
    # equal to 0 and level 1
    TSPRec(adj, curr_bound, 0, 1, curr_path, visited)

In [9]:
s=bandb(n)

In [11]:
#ofonction de visualisation sur la map 
def visualiser_sur_map(List):
    f=pd.read_csv("C:\\Users\\WIAM\\OneDrive\\Documents\\ors.csv").to_numpy() #on relit le contenu du csv fournit 
    d=[]
    for i in range (len(f)):# on remplit une nouvelle liste avec des listes contenant chacune l'indice de la localisation ainsi que ces coordonés 
        d+=[[f[i][1],f[i][2],f[i][3]]]
        lst=[]

    for l in List :
        for i in range(len(d)):
            if l==int(d[i][0]):
                lst+=[tuple([d[i][2],d[i][1]])] #on fait traduire le chemin obtenu et qui donne en output des indices des villes , avec des tuples de coordonnées pour les donner à l'api de la map 
    m = folium.Map(location=[34.00106,-6.78028],zoom_start=5) #on initie la librairie follium sur le maroc 

    loc=lst

    folium.PolyLine(loc,
                color='blue',
                weight=2,
                opacity=0.8).add_to(m) #on donne à follium la liste du chemin en coordonnées obtenu et trace le chemin sur la map 
    return m

In [13]:
visualiser_sur_map(s) #on appelle la focntion 